In [11]:
from keras import layers
from keras.models import Sequential
import keras_tuner as kt
from tensorflow.keras import optimizers


In [12]:
def build_model(hp):

    model = Sequential()
    
    model.add(layers.Conv2D(32,(3,3),activation="relu", input_shape=(150,150,3)))
    model.add(layers.MaxPooling2D((2,2)))
    
    model.add(layers.Conv2D(64,(3,3),activation="relu", input_shape=(150,150,3)))
    model.add(layers.MaxPooling2D((2,2)))
    
    model.add(layers.Conv2D(128,(3,3), activation="relu", input_shape=(150,150,3)))
    model.add(layers.MaxPooling2D((2,2)))
    
    model.add(layers.Conv2D(128,(3,3),activation="relu", input_shape=(150,150,3)))
    model.add(layers.MaxPooling2D((2,2)))
    
    model.add(layers.Flatten())

    #hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp.Choice("layer size", [128,256,512]), activation="relu"))
    model.add(layers.Dense(1, activation="sigmoid"))

    model.compile(loss="binary_crossentropy", optimizer=optimizers.RMSprop(learning_rate=1e-4),metrics=["acc"])
    return model


In [13]:
keras_tuner=kt.RandomSearch(build_model, objective="val_acc", overwrite=True, max_trials=3, 
                            directory="models/parameter_tuning", project_name="parameter_tuning")

In [14]:
train_directory="datasets/catdog/train"
validation_directory="datasets/catdog/validation"

In [15]:
#data generation
from keras.preprocessing.image import ImageDataGenerator
import os

train_datagen=ImageDataGenerator(rescale=1./255)
validation_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(train_directory, target_size=(150,150), batch_size=20, class_mode="binary")
validation_generator=validation_datagen.flow_from_directory (validation_directory, target_size=(150,150), batch_size=20, class_mode="binary")


Found 6400 images belonging to 2 classes.
Found 1600 images belonging to 2 classes.


In [ ]:
keras_tuner.search_space_summary()
keras_tuner.search(train_generator, validation_data=validation_generator, epochs=25)
best_model=keras_tuner.get_best_models()[0]

Trial 1 Complete [00h 39m 05s]
val_acc: 0.8137500286102295

Best val_acc So Far: 0.8137500286102295
Total elapsed time: 00h 39m 05s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
256               |512               |layer size

Epoch 1/25
320/320 [==============================] - 85s 265ms/step - loss: 0.6603 - acc: 0.5895 - val_loss: 0.6459 - val_acc: 0.5938
Epoch 2/25
320/320 [==============================] - 84s 263ms/step - loss: 0.5907 - acc: 0.6809 - val_loss: 0.5711 - val_acc: 0.6938
Epoch 3/25
320/320 [==============================] - 85s 267ms/step - loss: 0.5442 - acc: 0.7283 - val_loss: 0.5384 - val_acc: 0.7150
Epoch 4/25
320/320 [==============================] - 84s 264ms/step - loss: 0.5043 - acc: 0.7536 - val_loss: 0.5514 - val_acc: 0.7031
Epoch 5/25
320/320 [==============================] - 84s 264ms/step - loss: 0.4715 - acc: 0.7747 - val_loss: 0.4651 - val_acc: 0.7788
Epoch 6/25
320/320 [==============================] - 86s 268ms